## Adding Data Frames to Redis
PyArrow serializes the Data Frame to add to Redis as a whole piece

In [1]:
import pandas as pd
df = pd.read_csv("the_office_series.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    188 non-null    int64  
 1   Season        188 non-null    int64  
 2   EpisodeTitle  188 non-null    object 
 3   About         188 non-null    object 
 4   Ratings       188 non-null    float64
 5   Votes         188 non-null    int64  
 6   Viewership    188 non-null    float64
 7   Duration      188 non-null    int64  
 8   Date          188 non-null    object 
 9   GuestStars    29 non-null     object 
 10  Director      188 non-null    object 
 11  Writers       188 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 17.8+ KB


In [3]:
import redis
r = redis.Redis(host='localhost', port=6379, db=0)

In [4]:
import pyarrow as pa

context = pa.default_serialization_context()
r.set("key",context.serialize(df).to_buffer().to_pybytes())

<ipython-input-4-30a97950b199>:3: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pa.default_serialization_context()


True

In [5]:
df2 = context.deserialize(r.get("key"))
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    188 non-null    int64  
 1   Season        188 non-null    int64  
 2   EpisodeTitle  188 non-null    object 
 3   About         188 non-null    object 
 4   Ratings       188 non-null    float64
 5   Votes         188 non-null    int64  
 6   Viewership    188 non-null    float64
 7   Duration      188 non-null    int64  
 8   Date          188 non-null    object 
 9   GuestStars    29 non-null     object 
 10  Director      188 non-null    object 
 11  Writers       188 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 17.8+ KB


## JSON data and Redis
If the dataframe is small, you can convert it to a json file to add to Redis

In [6]:
r = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)
data = {'Product': ['Desktop Computer','Tablet','iPhone','Laptop'],
        'Price': [700,250,800,1200]
        }

df3 = pd.DataFrame(data, columns= ['Product', 'Price'])
df3

,Product,Price
0,Desktop Computer,700
1,Tablet,250
2,iPhone,800
3,Laptop,1200


In [7]:
import json
result = df3.to_json(orient="index")
parsed = json.loads(result)

In [8]:
r.set("df3",json.dumps(parsed))

True

In [9]:
df4 = r.get("df3")
df4

'{"0": {"Product": "Desktop Computer", "Price": 700}, "1": {"Product": "Tablet", "Price": 250}, "2": {"Product": "iPhone", "Price": 800}, "3": {"Product": "Laptop", "Price": 1200}}'

In [10]:
df4 = pd.read_json(df4, orient='index')

In [11]:
df4

,Product,Price
0,Desktop Computer,700
1,Tablet,250
2,iPhone,800
3,Laptop,1200


## Dictionaries into Redis

You can also transform your dataframe into a dictionary and save it to redis as key/value pairs

In [12]:
data2 = {'Menu': ['Burger','MacNCheese', 'Salad','Lamb Chops','Quesadilla'],
        'Price': [10,12,9,13,8]
        }

df5 = pd.DataFrame(data2, columns= ['Menu', 'Price'])
df5

,Menu,Price
0,Burger,10
1,MacNCheese,12
2,Salad,9
3,Lamb Chops,13
4,Quesadilla,8


In [13]:
df5=df5.to_dict()


In [14]:
r.set(str(df5.keys()),str(df5.values()))


True

In [24]:
df6 = r.get(str(df5.keys()))
df6

"dict_values([{0: 'Burger', 1: 'MacNCheese', 2: 'Salad', 3: 'Lamb Chops', 4: 'Quesadilla'}, {0: 10, 1: 12, 2: 9, 3: 13, 4: 8}])"

In [23]:
df6
df6 = df6[11::]

res = eval(df6)
df6

"([{0: 'Burger', 1: 'MacNCheese', 2: 'Salad', 3: 'Lamb Chops', 4: 'Quesadilla'}, {0: 10, 1: 12, 2: 9, 3: 13, 4: 8}])"

In [17]:
res
df6 = pd.DataFrame(res)

In [18]:
df6 = df6.transpose()
df6 = df6.rename( columns={0:"Menu",1:"Price"})

In [19]:
df6

,Menu,Price
0,Burger,10
1,MacNCheese,12
2,Salad,9
3,Lamb Chops,13
4,Quesadilla,8
